In [1]:
import os
import numpy as np
from astropy.cosmology import Planck15 as cosmo
from astrotog import functions as afunc
from astrotog import classes as aclasses
from astrotog import top_level_classes as atopclass
from mpi4py import MPI
import multiprocessing as mp
from itertools import repeat
import datetime
import time
from copy import copy, deepcopy
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
# Set seed for reproduceability
np.random.seed(12345)

/Users/cnsetzer/miniconda3/envs/astrotog/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/cnsetzer/miniconda3/envs/astrotog/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# ---------------------------------------------------------------
# Section that user can edit to tailor simulation on global level
# ---------------------------------------------------------------
batch_size = 50
batch_mp_workers = 2
verbose = True
# batch_size = 'all'

t_start = time.time()
os.system('clear')

# -------------------------------------------------------------------
# Section that user can edit to tailor simulation on primary process
# -------------------------------------------------------------------
dithers = False
survey_version = 'lsstv4'
add_dithers = False
t_before = 40.0
t_after = 40.0

seds_path = '/Users/cnsetzer/Documents/LSST/sedb/rosswog/NSNS/winds'
cadence_path = \
    '/Users/cnsetzer/Documents/LSST/surveydbs/kraken_2036.db'
throughputs_path = '/Users/cnsetzer/Documents/LSST/throughputs/lsst'
reference_flux_path = '/Users/cnsetzer/Documents/LSST/throughputs/references'
efficiency_table_path = '/Users/cnsetzer/Documents/LSST/Cadence/LSSTmetrics/example_data/SEARCHEFF_PIPELINE_DES.DAT'
run_dir = 'lsst_sim_run_kraken2036_' + datetime.datetime.now().strftime('%d%m%y_%H%M%S')
output_path = '/Users/cnsetzer/Documents/LSST/astrotog_output/' + run_dir + '/'

if not os.path.exists(output_path):
    os.makedirs(output_path)
z_max = 0.04
num_processes = 1
z_bin_size = 0.02

transient_rate_gpc = 1000.0  # Currently this is the default
z_min = 0.0  # Currently this is the default in the simulation class
cadence_flags = 'combined'  # Currently use default in class
rate = 1000
# -----------------------------------------------
# -----------------------------------------------

sim_inst = atopclass.simulation(cadence_path=cadence_path,
                                throughputs_path=throughputs_path,
                                reference_path=reference_flux_path,
                                z_max=z_max, output_path=output_path,
                                cadence_flags=cadence_flags,
                                z_min=z_min, z_bin_size=z_bin_size,
                                multiproc=False,
                                num_processes=num_processes,
                                batch_size=batch_size, cosmology=cosmo,
                                rate_gpc=rate, dithers=dithers,
                                simversion=survey_version,
                                add_dithers=add_dithers,
                                t_before=t_before, t_after=t_after,
                                response_path=efficiency_table_path)
LSST_survey = atopclass.LSST(sim_inst)

 reading from database sqlite:////Users/cnsetzer/Documents/LSST/surveydbs/kraken_2036.db
Not doing all observations here 
5, 3 combined
SELECT * FROM SummaryAllProps WHERE proposalId in (5, 3)
Reading in raw tables successful
replacing names works


/Users/cnsetzer/miniconda3/envs/astrotog/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/cnsetzer/.local/lib/python3.5/site-packages/opsimsummary-1.19.7-py3.5.egg/opsimsummary/opsim_out.py:574: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.sort_values(by='expMJD', inplace=True)


dropping duplicates works
358.475901
dithercolumns good!


/Users/cnsetzer/.local/lib/python3.5/site-packages/opsimsummary-1.19.7-py3.5.egg/opsimsummary/opsim_out.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.ditheredRA[mask] = df.ditheredRA[mask] - maxval


1846288 1846288
join good!
joining dithers works

Changing units for lsstv4 from degrees
check that the format matches the expectation


In [3]:
stored_obs_data = pd.read_csv('/Users/cnsetzer/Documents/LSST/astrotog_output/lsst_sim_run_kraken2036_210818_151053/observations.csv', index_col = 0)
stored_param_data = pd.read_csv('/Users/cnsetzer/Documents/LSST/astrotog_output/lsst_sim_run_kraken2036_210818_151053/parameters.csv', index_col = 0)
stored_other_obs_data = pd.read_csv('/Users/cnsetzer/Documents/LSST/astrotog_output/lsst_sim_run_kraken2036_210818_151053/other_observations.csv', index_col = 0)

In [4]:
def process_nightly_coadds(obs_df, survey):
    """
    Function which coadds all observations within 12 hours of each other but
    avoids reprocessing observations more than once.

    """
    coadded_df = pd.DataFrame(columns=obs_df.columns)
    new_coadded = pd.DataFrame(columns=['coadded night'])
    coadded_df = coadded_df.join(new_coadded)

    for transient in list(obs_df['transient_id'].unique()):
        t_obs_df = obs_df.query('transient_id == {}'.format(transient))
        for band in list(t_obs_df['bandfilter'].unique()):
            band_df = t_obs_df.query('bandfilter == \'{}\''.format(band))
            coadded_indicies = []
            for iter, row in band_df.iterrows():
                if iter in coadded_indicies:
                    continue
                else:
                    same_night_df = band_df[(band_df['mjd'] - row['mjd'] > -0.5) & (band_df['mjd'] - row['mjd'] < 0.5)]
                    
                    coadding_series = deepcopy(row)
                    coadd_band = 'lsst{}'.format(band)
                    if len(same_night_df.index) > 1:
                        coadding_series['mjd'] = same_night_df['mjd'].mean()
                        coadding_series['extincted flux'] = same_night_df['extincted flux'].mean()
                        coadding_series['source flux'] = same_night_df['source flux'].mean()
                        coadding_series['instrument flux'] = same_night_df['instrument flux'].mean()
                        coadding_series['instrument flux one sigma'] = np.sqrt(np.sum(np.square(same_night_df['instrument flux one sigma'])))/len(same_night_df.index)
                        coadding_series['source flux one sigma'] = np.sqrt(np.sum(np.square(same_night_df['source flux one sigma'])))/len(same_night_df.index)
                        coadding_series['extincted flux one sigma'] = np.sqrt(np.sum(np.square(same_night_df['extincted flux one sigma'])))/len(same_night_df.index)
                        coadding_series['airmass'] = same_night_df['airmass'].mean()
                        coadding_series['five sigma depth'] = same_night_df['five sigma depth'].mean()
                        coadding_series['lightcurve phase'] = same_night_df['lightcurve phase'].mean()
                        coadding_series['extincted magnitude'] = afunc.flux_to_mag(coadding_series['extincted flux'], survey.reference_flux_response[coadd_band])
                        coadding_series['source magnitude'] = afunc.flux_to_mag(coadding_series['source flux'], survey.reference_flux_response[coadd_band])
                        coadding_series['instrument magnitude'] = afunc.flux_to_mag(coadding_series['instrument flux'], survey.reference_flux_response[coadd_band])
                        coadding_series['instrument mag one sigma'] = afunc.magnitude_error(coadding_series['instrument flux'], coadding_series['instrument flux one sigma'], survey.reference_flux_response[coadd_band])
                        coadding_series['extincted mag one sigma'] = afunc.magnitude_error(coadding_series['extincted flux'], coadding_series['extincted flux one sigma'], survey.reference_flux_response[coadd_band])
                        coadding_series['source mag one sigma'] = afunc.magnitude_error(coadding_series['source flux'], coadding_series['source flux one sigma'], survey.reference_flux_response[coadd_band])
                        coadding_series['signal_to_noise'] = coadding_series['instrument flux']/coadding_series['instrument flux one sigma']
                        coadding_series['coadded night'] = True
                    else:
                        coadding_series['coadded night'] = False
                    coadded_indicies.extend(list(same_night_df.index))
                    coadded_df = coadded_df.append(coadding_series, ignore_index=True)

    return coadded_df

In [5]:
# Detections
if verbose:
    print('\nNow processing for detections.')
filters = {'snr': {'type': 'value',
                   'num_count': None,
                   'name': 'signal_to_noise',
                   'value': 0.25,
                   'gt_lt_eq': 'gt',
                   'absolute': True}
           # 'snr': {'type': 'value',
           #         'num_count': None,
           #         'name': 'signal_to_noise',
           #         'value': 5.0,
           #         'gt_lt_eq': 'gt',
           #         'absolute': False}
           }


Now processing for detections.


In [6]:
# Do coadds first
if verbose:
    print('Doing nightly coadds...')
coadded_observations = process_nightly_coadds(stored_obs_data, LSST_survey)

Doing nightly coadds...


In [7]:
if verbose:
    print('Processing coadded nights for transients alert triggers.')
coadded_observations = afunc.efficiency_process(LSST_survey, coadded_observations)

Processing coadded nights for transients alert triggers.


In [8]:
print(stored_obs_data)

     transient_id           mjd bandfilter  instrument magnitude  \
0               3  63291.201829          y             22.523019   
1               3  63293.295637          y             58.564914   
2               3  63302.107292          g             30.973780   
3               3  63302.125833          g             26.505843   
4               3  63304.100694          r             60.594439   
5               3  63304.117523          r             60.594439   
6               3  63305.131551          r             26.361418   
7               3  63305.152894          r             26.638257   
8               3  63307.101655          i             27.211378   
9               3  63307.114745          i             27.020818   
10              3  63312.143206          i             60.054990   
11              3  63312.162072          i             60.054990   
12              5  60531.405903          u             25.019232   
13              5  60531.425741          u      

In [9]:
# if verbose:
#     print('Processing coadded observations for ultra-low SNR.')
# coadded_observations = afunc.detect(coadded_observations, filters)

if verbose:
    print('Processing coadded observations for detections in line with Scolnic et. al 2018.')
detected_observations = afunc.scolnic_detections(stored_param_data, coadded_observations, stored_other_obs_data, LSST_survey)

Processing coadded observations for detections in line with Scolnic et. al 2018.


In [10]:
stored_param_data = afunc.param_observe_detect(stored_param_data, coadded_observations, detected_observations)


if verbose:
    t_end = time.time()
    print('\nSimulation completed successfully with elapsed time: {}.'.format(datetime.timedelta(seconds=int(t_end-t_start))))


Simulation completed successfully with elapsed time: 0:01:58.
